In [8]:
import socket, pandas as pd
from ipynb.fs.full.df_collapser import collapse

In [2]:
def recv_msg(sock):
    # Read message length and unpack it into an integer
    msglen = int(sock.recv(8))
    if not msglen:
        return None
    # Read the message data
    return recvall(sock, msglen)

def recvall(sock, n):
    # Helper function to recv n bytes or return None if EOF is hit
    data = bytearray()
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data.extend(packet)
    return data

In [3]:
def dataframe_from_string(string):
    samples=string.split("|")
    if len(samples) == 6:
        lin_acc_x=list(map(float, samples[0][1:len(samples[0])-1].split(", ")))
        lin_acc_y=list(map(float, samples[1][1:len(samples[1])-1].split(", ")))
        lin_acc_z=list(map(float, samples[2][1:len(samples[2])-1].split(", ")))
        rot_x=list(map(float, samples[3][1:len(samples[3])-1].split(", ")))
        rot_y=list(map(float, samples[4][1:len(samples[4])-1].split(", ")))
        rot_z=list(map(float, samples[5][1:len(samples[5])-1].split(", ")))
        df_len=min([len(lin_acc_x), len(lin_acc_y), len(lin_acc_z), len(rot_x), len(rot_y), len(rot_z)])

        df=pd.DataFrame(columns=["rotationRate.x", "rotationRate.y", "rotationRate.z", "userAcceleration.x", "userAcceleration.y", "userAcceleration.z"])
        df["rotationRate.x"]=rot_x[0:df_len]
        df["rotationRate.x"]=df["rotationRate.x"]*2
        df["rotationRate.y"]=rot_y[0:df_len]
        df["rotationRate.y"]=df["rotationRate.y"]*2
        df["rotationRate.z"]=rot_z[0:df_len]
        df["rotationRate.z"]=df["rotationRate.z"]*2
        df["userAcceleration.x"]=lin_acc_x[0:df_len]
        df["userAcceleration.x"]=df["userAcceleration.x"]/5
        df["userAcceleration.y"]=lin_acc_y[0:df_len]
        df["userAcceleration.y"]=df["userAcceleration.y"]/5
        df["userAcceleration.z"]=lin_acc_z[0:df_len]
        df["userAcceleration.z"]=df["userAcceleration.z"]/5
        return df

In [4]:
HOST = ''
PORT = 1060

# the function expects that each n seconds the phone passes all the readings coming from the sensors
# into a string

def phone_comm(host, port, pipeline):

    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
    addr=(host, port)
    s.bind(addr)
    s.listen(1)
    print ('the server is listening at',s.getsockname())
    ss,sockname = s.accept()
    while True:
        #accept the complete message
        msg=recv_msg(ss).decode()
        # obtain the dataframe from the string
        df=dataframe_from_string(msg)
        # collapse the dataset into windows
        windowed_df=collapse(df)
        # predict the activity from the collapsed df
        prediction=pipeline.predict(windowed_df)
        # print the prediction
        print(prediction)

    ss.close()